<a href="https://colab.research.google.com/github/fionatjahjono/ml_project/blob/main/kmeans_clustering_ps133.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
app = Flask(__name__)

def load_and_preprocess_data(user_file, event_file):
    # Load user and event data from csv files
    user_data = pd.read_csv(user_file)
    event_data = pd.read_csv(event_file)

    # Convert 'free' to 0 in event_price
    event_data["event_price"] = event_data["event_price"].replace('free', 0)

    # Convert event_price to numeric
    event_data["event_price"] = pd.to_numeric(event_data["event_price"], errors='coerce')

    # Drop rows with NaN values in event_price
    event_data = event_data.dropna(subset=["event_price"])

    # Convert latitude and longitude to numeric, coercing non-numeric values to NaN
    user_data[["latitude", "longitude"]] = user_data[["latitude", "longitude"]].apply(pd.to_numeric, errors='coerce')
    event_data[["latitude", "longitude"]] = event_data[["latitude", "longitude"]].apply(pd.to_numeric, errors='coerce')

    # Drop rows with NaN values in latitude or longitude
    user_data = user_data.dropna(subset=["latitude", "longitude"])
    event_data = event_data.dropna(subset=["latitude", "longitude"])

    # Normalize coordinates
    scaler = MinMaxScaler()
    user_data[["latitude", "longitude"]] = scaler.fit_transform(user_data[["latitude", "longitude"]])
    event_data[["latitude", "longitude"]] = scaler.transform(event_data[["latitude", "longitude"]])

    return user_data, event_data

In [ ]:
def calculate_distances(user, events):
    user_location = tf.constant([user["latitude"], user["longitude"]], dtype=tf.float32)
    event_locations = tf.constant(events[["latitude", "longitude"]].values, dtype=tf.float32)

    return tf.norm(user_location - event_locations, axis=1)

In [ ]:
def make_prediction(user, events, k):
    distances = calculate_distances(user, events)
    indices = np.argsort(distances)[:k]
    knn_events = events.iloc[indices]

    # Calculate weights based on distance
    weights = 1 / (np.array(distances)[indices] + 1e-10)  # Add a small constant to avoid division by zero

    # Perform weighted voting
    weighted_labels = knn_events["event_price"] * weights
    weighted_prediction = weighted_labels.sum() / weights.sum()

    return weighted_prediction, knn_events

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    # Get user_id from request
    user_id = request.json['user_id']

    # Define the value of k
    k = 5

    # Load and preprocess data
    user_data, event_data = load_and_preprocess_data("user_table.csv", "event_table.csv")

    # Get the user details based on user_id
    test_user = user_data[user_data["user_id"] == user_id].iloc[0]

    # Make prediction
    weighted_prediction, knn_events = make_prediction(test_user, event_data, k)

    # Prepare response
    response = {
        'weighted_prediction': weighted_prediction,
        'knn_events': knn_events.to_dict()
    }

    return jsonify(response)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# test data befor flask
k=5
user_data, event_data = load_and_preprocess_data("user_table.csv", "event_table.csv")

In [ ]:
# Get the user details based on user_id
user_id = 5  # replace 1 with the actual user_id you want to test
test_user = user_data[user_data["user_id"] == user_id].iloc[0]

In [ ]:
# Make prediction
weighted_prediction, knn_events = make_prediction(test_user, event_data, k)

In [ ]:
# Print the recommended events
print("\nWeighted Prediction for Event Price:")
print(weighted_prediction)

for i, event in knn_events.iterrows():
    print(f"Event {i+1}:")
    print(f"Name: {event['event_name']}")
    print(f"Location: {event['latitude']}, {event['longitude']}")
    print(f"Price: {event['event_price']}")
    print("------------------------")

print("\nEvents near the user:")
print(knn_events)